In [31]:
using LinearAlgebra,TimeIt

include("MatrixDisp.jl")

eye(n) = Matrix{Float64}(I,n,n)

eye (generic function with 1 method)

In [34]:
function LU!(A)
    n = size(A)[1]
    @assert n == size(A)[2] "The Matrix must be square matrix!"
    L = eye(n)
    U = zeros(n,n)
    U[1,1:n] = A[1,1:n]
    L[2:n,1] = A[2:n,1]/U[1,1]
    for r = 2:n
        U[r:r,r:n] = A[r:r,r:n] - L[r:r,1:r-1]*U[1:r-1,r:n]
        if r < n
            L[r+1:n,r:r] = (A[r+1:n,r:r] - L[r+1:n,1:r-1]*U[1:r-1,r:r])/U[r,r]
        end
    end
    return L,U
end

LU! (generic function with 1 method)

In [32]:
@latex A = [1 2 3;2 5 2;3 1 5]

HTML{String}("\$\$\nA=\\begin{bmatrix}1&2&3\\\\2&5&2\\\\3&1&5\\end{bmatrix}\n\$\$")

In [33]:
@timeit L,U = A |> LU!;

1 loops, best of 3: 2.28 µs per loop


In [36]:
@latex L

HTML{String}("\$\$\nL=\\begin{bmatrix}1.0&0.0&0.0\\\\2.0&1.0&0.0\\\\3.0&-5.0&1.0\\end{bmatrix}\n\$\$")

In [37]:
@latex U

HTML{String}("\$\$\nU=\\begin{bmatrix}1.0&2.0&3.0\\\\0.0&1.0&-4.0\\\\0.0&0.0&-24.0\\end{bmatrix}\n\$\$")